#### Dataframes

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn as sk
import sklearn.tree as tree
from IPython.display import Image  
import pydotplus

In [36]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [37]:
dfe = pd.read_csv('Employment.csv')
dfp = pd.read_csv('Population.csv')
dfv = pd.read_csv('Vehicle.csv')
dfc = pd.read_csv('Vaccinations.csv')

C:\Users\ninja\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Cleaned Employment Dataframe

In [38]:
new_header = dfe.iloc[0] #Changes the first row to be the header of the dataframe
dfe = dfe[1:]
dfe.columns = new_header

In [39]:
#Counts how many columns in the table there are and drops all columns past "age over 75"
h = 0
for i in dfe.columns:
    if 'Margin of Error' in i:
        dfe.drop(columns = i, inplace = True)
    elif h > 12:
        dfe.drop(columns = i, inplace = True)
    else:
        h += 1

In [40]:
#Removes the id column
dfe.drop(columns = 'id', inplace = True)

In [41]:
#Renames the columns into simpler terms
dfe.columns = ['Zip Code',
              'Total Population Age 16+',
              'Ages 16-19',
              'Ages 20-24',
              'Ages 25-29',
              'Ages 30-34',
              'Ages 35-44',
              'Ages 45-54',
              'Ages 55-59',
              'Ages 60-64',
              'Ages 65-74',
              'Ages 75+']

In [42]:
#Defines a function to remove ZTCA5 from every zipcode
def Zipfinder(Zipcode):
        j = Zipcode.replace('ZCTA5 ', '')
        return j

In [43]:
dfe['Zip Code'] = dfe['Zip Code'].apply(Zipfinder) #Applies Zipfinder to the Zipcode column

In [44]:
dfe #Employment Dataframe

,Zip Code,Total Population Age 16+,Ages 16-19,Ages 20-24,Ages 25-29,Ages 30-34,Ages 35-44,Ages 45-54,Ages 55-59,Ages 60-64,Ages 65-74,Ages 75+
1,89010,410,8,0,8,34,69,43,26,75,102,45
2,89019,1730,60,95,219,141,117,339,280,101,320,58
3,89060,8347,144,349,608,271,689,1352,1018,873,1723,1320
4,89061,5394,196,182,109,389,396,508,458,835,1487,834
5,89439,1161,4,55,29,96,109,110,204,198,172,184
...,...,...,...,...,...,...,...,...,...,...,...,...
1765,96148,471,0,35,129,106,18,22,31,73,26,31
1766,96150,24911,1211,2431,2420,2482,3889,3218,2150,2605,3036,1469
1767,96155,0,0,0,0,0,0,0,0,0,0,0
1768,96161,14763,759,957,541,1353,3215,2746,1405,1186,2180,421


#### Cleaned Population Dataframe

In [45]:
#Converts Zip Code values into a string
def Zipconvert(Zipcode):
        z = str(Zipcode)
        return z

In [46]:
dfp['Zip Code'] = dfp['Zip Code'].apply(Zipconvert)

In [47]:
dfp.rename(columns = {'Population' : 'Total Population'}, inplace = True)

In [48]:
dfp #Population dataframe

,Zip Code,Total Population
0,90011,111165
1,91331,105458
2,90650,105304
3,90201,101965
4,90044,99443
...,...,...
1741,95680,10
1742,95430,7
1743,92267,6
1744,92364,6


#### Cleaned Vehicle Dataframe

In [49]:
dfv.drop(columns = ['Date', 'Duty', 'Fuel'], inplace = True) #Drops the columns date, duty, and fuel

In [50]:
dfv.drop(index = dfv[dfv['Zip Code'] == 'OOS'].index, inplace = True) #Drops all rows with OOS as the zip code value

In [51]:
dfv #Vehicle dataframe

,Zip Code,Model Year,Make,Vehicles
0,90001,2007,ACURA,15
1,90002,2007,ACURA,20
2,90003,2007,ACURA,29
3,90004,2007,ACURA,19
4,90006,2007,ACURA,15
...,...,...,...,...
602387,91405,Unk,Unk,36
602388,95816,Unk,Unk,217
602390,90012,Unk,Unk,12
602391,93065,Unk,Unk,11


In [52]:
dfvg = dfv.groupby('Zip Code')[['Make']].max() #Groups all vehicles by zip code and finds the most used make and stores it into dfvg
dfvg

,Make
Zip Code,
90000,WHITE
90001,VOLVO
90002,VOLVO
90003,VOLVO
90004,VOLVO
...,...
99836,OTHER/UNK
99840,OTHER/UNK
99900,OTHER/UNK


#### Cleaned Vaccinations Dataframe

#### Merged Dataframe

In [53]:
dff = dfe.merge(dfp, left_on = 'Zip Code', right_on = 'Zip Code') #Merges dfe and dfp into dff

In [54]:
dff = dff.merge(dfvg, left_on = 'Zip Code', right_on = 'Zip Code') #Merges dff and dfvg

In [55]:
dff = dff.set_index('Zip Code')

In [57]:
dff

,Total Population Age 16+,Ages 16-19,Ages 20-24,Ages 25-29,Ages 30-34,Ages 35-44,Ages 45-54,Ages 55-59,Ages 60-64,Ages 65-74,Ages 75+,Total Population,Make
Zip Code,,,,,,,,,,,,,
90001,42853,3928,4839,4848,4818,8082,6848,2953,2405,2694,1438,59832,VOLVO
90002,38146,3843,4911,4274,3431,7497,6413,2259,2179,2021,1318,53302,VOLVO
90003,52804,5135,5620,7236,5288,9767,8816,3536,2855,2782,1769,73730,VOLVO
90004,50096,2478,3993,6658,6487,9289,8219,3378,3050,4151,2393,60541,VOLVO
90005,33217,1528,2651,4695,3965,6750,4835,2105,1661,2707,2320,39732,VOLVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96146,973,10,51,85,145,125,234,101,81,98,43,1145,VOLKSWAGEN
96148,471,0,35,129,106,18,22,31,73,26,31,580,Unk
96150,24911,1211,2431,2420,2482,3889,3218,2150,2605,3036,1469,29327,VOLVO
